# Modelo LSTM Complejo aplicando transformaciones sobre las variables cíclicas.
#### Este modelo LSTM busca predecir la intensidad y la velocidad media de los vehículos en un punto dada una entrada de datos.

El primer paso es importar todas las librerías necesarias para el correcto funcionamiento de este cuaderno de Jupyter.

In [ ]:
# Librerías estándar
from math import sqrt  # Función matemática

# Manejo de datos
import numpy as np  # Manejo de arrays numéricos
import pandas as pd  # Manipulación de estructuras de datos (DataFrames)

# Visualización
import matplotlib.pyplot as plt  # Gráficas y visualización

# Preprocesamiento y métricas (Scikit-learn)
from sklearn.preprocessing import MinMaxScaler  # Normalización de datos
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  # Métricas de evaluación

# Modelado (Keras / TensorFlow)
import tensorflow as tf  # Backend de Keras, entrenamiento de modelos
from keras.models import Sequential  # Modelo secuencial
from tensorflow.keras.optimizers import Adam  # Optimizador Adam
from keras.layers import LSTM, Dense, Dropout  # Capas de red neuronal
from keras.regularizers import l2  # Regularización L2

# Guardado/carga de modelos y objetos
import joblib  # Serialización de modelos/objetos

Cargamos los datos sobre los que se va a entrenar el modelo.

In [ ]:
id_objetivo = 3498 # <--- Sustituir este valor según el objetivo
df = pd.read_csv(f'../data/{id_objetivo}.csv')

# Mostrar las primeras filas del DataFrame
print(f"Datos cargados para el ID objetivo {id_objetivo}:")
df.head()

### Preprocesamiento de datos.

Pasamos la columna `error` a numérica. La columna `tipo_elem` no la modificamos ya que no se va a contar con ella para el entrenamiento, pues su valor es el mismo todo el rato (estos datos corresponden al mismo elemento todo el rapo, por lo que el tipo de elemento será siempre el mismo).

In [ ]:
df['error'] = (df['error'] != 'N').astype(int)

# Vemos si se ha aplicado correctamente la transformación
df.head()

El siguiente paso es transformar las variables cíclicas a través de las funciones seno y coseno para capturar la periodicidad de las características temporales. En este caso, se ha calculado el ciclo diario, semanal, mensual y anual.

In [ ]:
# Convertir la columna 'fecha' a tipo datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Extraer características de la fecha
# Hora exacta en decimal
df['hora'] = df['fecha'].dt.hour + df['fecha'].dt.minute / 60.0
# Día de la semana
df['day_of_week'] = df['fecha'].dt.dayofweek  # Lunes=0, Domingo=6
# Mes del año
df['month'] = df['fecha'].dt.month # 1 a 12
# Día del año
df['day_of_year'] = df['fecha'].dt.dayofyear 


# Aplicar las funciones seno y coseno
# Ciclo diario
df['hora_sin'] = np.sin(2 * np.pi * df['hora'] / 24)
df['hora_cos'] = np.cos(2 * np.pi * df['hora'] / 24)
# Ciclo semanal
df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
# Ciclo mensual
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
# Ciclo anual (día del año)
df['doy_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
df['doy_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)


# Verificar que las nuevas columnas se han añadido correctamente
df.head()

El siguiente paso es verificar que no haya ninguna columna con algún tipo de dato que no queramos, por lo que se imprimen los tipos de dato de los valores de cada columna.

In [ ]:
df.dtypes

Ya empezamos a prepararnos para entrenar el modelo. Se eliminan las filas nulas y se filtra el DataFrame para que solo tenga las columnas que se vayan a emplear para entrenar el modelo, poniendo al final las variables que se van a predecir.

In [ ]:
df = df.dropna()
df = df[['ocupacion', 'carga', 'error', 'periodo_integracion', 'hora_sin', 'hora_cos', 
         'dow_sin', 'dow_cos', 'doy_sin', 'doy_cos', 'month_sin', 'month_cos', 'intensidad', 'vmed']]

Escalamos los datos para que todas las variables tengan la misma importancia y así el modelo pueda aprender de manera más eficiente.

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(df)

Creamos secuencias de datos, un proceso importante porque el valor futuro depende del orden y los valores pasados, y de esta manera se captura la dinámica temporal del fenómeno. En este paso también se separan las variables objetivo del resto de columnas.

Para cada secuencia se calculan 96 pasos, al ser los datos cada 15 minutos, esto implica que cada secuencia subre 1 día completo de datos.

In [ ]:
def create_sequences(data, sequence_length=96):
    X, y = [], []
    for i in range(sequence_length, len(data)):
        X.append(data[i-sequence_length:i, :-2])  # input features
        y.append(data[i, -2:])  # target: [intensidad, vmed]
    return np.array(X), np.array(y)

sequence_length = 96
X, y = create_sequences(scaled, sequence_length)

Luego se divide el conjunto de datos en dos: el 80 % se destina al entrenamiento mientras que el 20 % a la validación.

In [ ]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

### Modelo LSTM.

Ahora ya se monta la arquitectura del modelo LSTM y se entrena el modelo.

In [ ]:
model = Sequential()

# Primera capa LSTM
model.add(LSTM(128, return_sequences=True, 
               dropout=0.1, recurrent_dropout=0, 
               kernel_regularizer=l2(1e-5), 
               input_shape=(X.shape[1], X.shape[2])))

# Segunda capa LSTM
model.add(LSTM(64, return_sequences=False, 
               dropout=0.1, recurrent_dropout=0, 
               kernel_regularizer=l2(1e-5)))

# Capa densa oculta 1
model.add(Dense(64, activation='relu', kernel_regularizer=l2(1e-5)))
model.add(Dropout(0.1))

# Capa densa oculta 2
model.add(Dense(32, activation='relu', kernel_regularizer=l2(1e-5)))
model.add(Dropout(0.1))

# Capa de salida
model.add(Dense(2))  # intensidad y vmed

# Compilación del modelo
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_test, y_test))

### Predicciones.

Una vez entrenado el modelo, se hacen predicciones.

In [ ]:
y_pred = model.predict(X_test)

Y se deshace el escalado que se hizo al principio sobre las variables predichas.

In [ ]:
def invert_scale(scaled_values, original_data, columns_idx):
    unscaled = []
    for i in range(len(scaled_values)):
        row = np.zeros(original_data.shape[1])
        row[columns_idx[0]] = scaled_values[i][0]
        row[columns_idx[1]] = scaled_values[i][1]
        inv = scaler.inverse_transform([row])
        unscaled.append([inv[0][columns_idx[0]], inv[0][columns_idx[1]]])
    return np.array(unscaled)

y_pred_inv = invert_scale(y_pred, scaled, [-2, -1])
y_test_inv = invert_scale(y_test, scaled, [-2, -1])

### Evaluación del modelo.

Para evaluar el modelo calculamos las métricas MSE, MAE, RMSE y R².

In [ ]:
# 📊 METRICS
mse = mean_squared_error(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
rmse = sqrt(mse)
r2 = r2_score(y_test_inv, y_pred_inv)

print(f"MSE:  {mse:.2f}")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")

Otra buena manera de evaluar un modelo es viualizar gráficas.

El siguiente fragmento de código hace una comprativa de los valores reales y los predichos.

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(y_test_inv[:, 0], color='blue', label="Valores Reales")
plt.plot(y_pred_inv[:, 0], color='red', linestyle='--', label=f"Predicciones LSTM - RMSE: {rmse:.2f}")

# Confidence interval (70%)
residuals = y_test_inv[:, 0] - y_pred_inv[:, 0]
std_res = np.std(residuals)
upper = y_pred_inv[:, 0] + std_res
lower = y_pred_inv[:, 0] - std_res
plt.fill_between(range(len(y_pred_inv)), lower, upper, color='gray', alpha=0.3, label="Intervalo de Confianza (70%)")

plt.title("Predicciones del Modelo LSTM con Intervalo de Confianza del 70%")
plt.xlabel("Índice Temporal (15 min por paso)")
plt.ylabel("Intensidad")
plt.legend()
plt.show()

Se grafican los residuos para ver si los errores están centrados en cero y  de manera simétrica, lo cual indica que el modelo no tiene sesgos sistemáticos y está capturando bien la estructura de los datos.

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(residuals, color='red', label='LSTM Residuos')
plt.axhline(y=0, color='black', linestyle='--')
plt.title("Residuos del Modelo LSTM")
plt.xlabel("Índice Temporal (15 min por paso)")
plt.ylabel("Residuos")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(residuals, bins=30, color='salmon', label='LSTM Residuos')
plt.title("Histograma de Residuos del Modelo LSTM")
plt.xlabel("Residuos")
plt.ylabel("Frecuencia")
plt.legend()
plt.show()

Por último, se dibuja la curva de aprendizaje para ver cómo de bien ha aprendido el modelo.

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

### Guardar modelo.

El último paso es guardar el modelo entrenado y el escalador para poder usarlos cuando sea necesario, sin tener que ejecutar todo este cuaderno de nuevo.

In [ ]:
model.save("./saved/lstm_model_cyclical.h5")
joblib.dump(scaler, "./saved/lstm_scaler_cyclical.pkl")

print("Modelo y scaler guardados correctamente.")